## Package

In [4]:
import numpy as np
import os
import argparse
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 
from sklearn.metrics import classification_report
import pickle
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

## Config

In [5]:
parser = {
    "experiment_title": "dog-cat-test",
    "root": "../",
    "seeds": 123,
    "batch_size": 16,
    "class_a_index": 5, # 5 -> dog
    "class_b_index": 3, # 3 -> cat
    "class_a_size": None, # Use None to read all class_a samples (5000 rows)
    "class_b_size": None,
    "class_a_weight": 256,
    "class_b_weight": 1,
    "epoch": 20,
    "download_cifar10": True, # set to True if you have not downloaded cifar10 dataset 
    "lr": 0.1, # learning rate
    "use_batchnorm": True,
    "num_classes": 10,
    "l2_penalty": 0.0,
}
config = argparse.Namespace(**parser)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Reprodicibility

In [6]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    
set_seed(config.seeds)

## Load Data

In [7]:
class CustomDataLoader:
    
    def __init__(self, class_a_size=None, class_b_size=None, seeds=123, download=True):
        """
        Arguments:
            class_a_size: The number of samples for class a (e.g. dog).
            class_b_size: The number of samples for class b (e.g. cat).
        """
        self.class_a_size = class_a_size # e.g. 5000
        self.class_b_size = class_b_size # e.g. 5000
        self.seeds = seeds
        self.classes = ['plane', 'car', 'bird', 'cat',
                   'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        self.download = download
        
    def sampling(self, indices, num, seeds=123):
        """
        Sample num samples from indices randomly. If num is a float and 0.0 <= num <= 1.0,
        we sample num percentage of samples from indices.
        """
        np.random.seed(self.seeds)
        if isinstance(num, float) and 0 <= num <= 1:
            size = int(num * len(indices))
            samples = np.random.choice(indices, size)
        elif isinstance(num, int) and num <= len(indices):
            size = num
            samples = np.random.choice(indices, size)
        elif num is None:
            samples = indices
        else:
            print("Please make sure 'num' is in the correct range")
        return samples       
        
    def get_subset_index(self, labels, targets):
        
        # get subsets
        sample_indices = []
        targets = set(targets)
        for i in range(len(labels)):
            if labels[i] in targets:
                sample_indices.append(i)
            
        return sample_indices

    def load_dataset(self, data_dir, batch_size, train_a_label=5, train_b_label=3):
        
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        # 1. read train data belonged to train_labels
        trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True,
                                                download=self.download, transform=transform)
        
        # 1-1. get index belonged to class a and b with sampling
        class_a_indices = self.get_subset_index(trainset.targets, [train_a_label])
        class_a_indices = self.sampling(class_a_indices, self.class_a_size, self.seeds)
        
        class_b_indices = self.get_subset_index(trainset.targets, [train_b_label])
        class_b_indices = self.sampling(class_b_indices, self.class_b_size, self.seeds)
        
        train_index_subset = np.concatenate((class_a_indices, class_b_indices)).tolist()
        trainsubset = torch.utils.data.Subset(trainset, train_index_subset)

        # 1. read test data and separate them into (dog, cat) and (the other 8)
        testset = torchvision.datasets.CIFAR10(root=data_dir, train=False,
                                               download=self.download, transform=transform)
        test_index_class_ab = self.get_subset_index(
            labels=testset.targets,
            targets=[train_a_label, train_b_label]
        )
        test_index_others = self.get_subset_index(
            labels=testset.targets,
            targets=[cls for cls in range(len(self.classes)) if cls not in [train_a_label, train_b_label]]
        )
        testsubset_ab = torch.utils.data.Subset(testset, test_index_class_ab)
        testsubset_others = torch.utils.data.Subset(testset, test_index_others)

        return trainsubset, (testset, testsubset_ab, testsubset_others)

    def generate_loader(self, dataset, batch_size, shuffle=True, drop_last=True):
        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            drop_last=True
        )
        return dataloader

In [8]:
mydataloader = CustomDataLoader(
    class_a_size=config.class_a_size,
    class_b_size=config.class_b_size,
    seeds=config.seeds,
    download=config.download_cifar10)

trainset, (testset, testset_ab, testset_others) = mydataloader.load_dataset(
    data_dir=os.path.join(config.root, "data"),
    batch_size=config.batch_size,
    train_a_label=config.class_a_index, # dog
    train_b_label=config.class_b_index, # cat
)

train_loader = mydataloader.generate_loader(
    dataset=trainset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True,
)

test_loader = mydataloader.generate_loader(
    dataset=testset,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=False,
)

test_ab_loader = mydataloader.generate_loader(
    dataset=testset_ab,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=False,
)

test_others_loader = mydataloader.generate_loader(
    dataset=testset_others,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=False,
)

Files already downloaded and verified
Files already downloaded and verified


## Model Architecture

### Customized Convolutional Networks

In [33]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv_layers1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.conv_layers2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.dense_layers = nn.Sequential(
            nn.Linear(128*4*4, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
        )
        self.fc = nn.Linear(128, num_classes)
    def forward(self, x):
        out = self.conv_layers1(x)
        out = self.conv_layers2(out)
        out = out.view(out.size(0), -1) # batch x 128 x 4 x 4 -> batch x (128*4*4)
        out = self.dense_layers(out)
        out = self.fc(out)
        return out

### ResNet

In [10]:
def conv3x3(in_channels, out_channels, stride=1, kernel=3):
    return nn.Conv2d(in_channels, out_channels, kernel_size=kernel, 
                     stride=stride, padding=1, bias=False)

In [11]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, use_batchnorm=True):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        self.use_batchnorm = use_batchnorm
        
    def forward(self, x):
        residual = x
        
        if self.use_batchnorm:
            out = self.conv1(x)
            out = self.bn1(out)
            out = self.relu(out)
            out = self.conv2(out)
            out = self.bn2(out)
        else:
            out = self.conv1(x)
            out = self.relu(out)
            out = self.conv2(out)
            
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


In [28]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10, use_batchnorm=True):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.use_batchnorm = use_batchnorm
        self.conv = conv3x3(3, 64, kernel=5)
        self.bn = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 64, layers[0])
        self.layer2 = self.make_layer(block, 128, layers[1], 2)
        self.layer3 = self.make_layer(block, 256, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(256, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            if self.use_batchnorm:
                downsample = nn.Sequential(
                    conv3x3(self.in_channels, out_channels, stride=stride),
                    nn.BatchNorm2d(out_channels))
            else:
                conv3x3(self.in_channels, out_channels, stride=stride)
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample, use_batchnorm=self.use_batchnorm))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels, use_batchnorm=self.use_batchnorm))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        
        if self.use_batchnorm:
            out = self.bn(out)
            
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

## Train Test Functions

### Utils


In [13]:
def logging(message, path, mode="w+"):
    print(message)
    message += "\n"
    with open(path, mode) as file:
        file.write(message)

### Train

In [14]:
def train(trainloader, testloaders, criterion, optimizer, config=config):
    
    # Record training process
    logging(
        message="Start training the experiment: {}!".format(config.experiment_title),
        path=os.path.join(config.root, "results/logs", config.experiment_title+".txt"),
        mode="w+",
    )
    
    # Start training
    total_step = len(trainloader)
    num_epochs = config.epoch
    fractions = []
    for epoch in range(num_epochs):
        model.train()
        for i, (images, labels) in enumerate(trainloader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                message = "Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}" \
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item())
                logging(
                    message=message,
                    path=os.path.join(config.root, "results/logs", config.experiment_title+".txt"),
                    mode="a+",
                )
                
        # After each epoch, we evaluate on "cat and dog test images" and "test images from the other 8 classes"
        fractions_catdog_other8 = []
        for testloader in testloaders:
            evaluation_results = evaluation(model, testloader)
            fractions_catdog_other8.append(evaluation_results["fraction_of_class_a"])
        fractions.append(fractions_catdog_other8)
        
        # We save fraction of dogs after each epoch in case that we will stop the training process earlier
        fractions_path=os.path.join(config.root, "results/fractions", config.experiment_title+".pkl")
        with open(fractions_path, "wb") as file:
            pickle.dump(fractions, file) 
    
    return model

### Test

In [15]:
def evaluation(model,
               testloader,
               config=config):
    model.eval()
    with torch.no_grad():
        y_trues, y_preds = [], []
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            y_preds += predicted.tolist()
            y_trues += labels.tolist()
        report = classification_report(y_trues, y_preds, zero_division=0)
    
    fraction_of_class_a = sum([config.class_a_index==y_pred for y_pred in y_preds])/len(y_preds)
    
    # save results
    evaluation_results = {
        "y_trues": y_trues,
        "y_preds": y_preds, 
        "report": report,
        "fraction_of_class_a": fraction_of_class_a,
    }
    return evaluation_results

## Start Training

The training process will be saved into a text file, named by config.experiment_title, in **results/logs/**. Also, the fraction of dogs will be saved as a pickle file in **results/fractions/**.

In [34]:
# requirement
model = ResNet(
    block=ResidualBlock,
    layers=[2, 2, 2],
    num_classes=config.num_classes,
    use_batchnorm=config.use_batchnorm
).to(device)

# model = CustomCNN(config.num_classes).to(device)
weights = torch.ones(10)
weights[config.class_a_index] = config.class_a_weight
weights[config.class_b_index] = config.class_b_weight
criterion = nn.CrossEntropyLoss(weight=weights.to(device))

learning_rate = config.lr
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=config.l2_penalty)

# train
model_trained = train(train_loader, [test_ab_loader, test_others_loader], criterion, optimizer)

# save model
torch.save(
    model_trained.state_dict(),
    os.path.join(config.root, "results/models", config.experiment_title+".ckpt")
)

Start training the experiment: dog-cat-test!
Epoch [1/20], Step [100/625] Loss: 0.0219
Epoch [1/20], Step [200/625] Loss: 0.0430
Epoch [1/20], Step [300/625] Loss: 0.0175
Epoch [1/20], Step [400/625] Loss: 0.0236
Epoch [1/20], Step [500/625] Loss: 0.0301
Epoch [1/20], Step [600/625] Loss: 0.0555


KeyboardInterrupt: 

### Evaluation

In [ ]:
results = evaluation(model_trained, test_loader)
print(results["report"])

print("---"*3)

results = evaluation(model_trained, test_ab_loader)
print(results["report"])

### Reload Fractions

In [ ]:
fractions_path=os.path.join(config.root, "results/fractions", config.experiment_title+".pkl")
with open(fractions_path, "rb") as file:
    fractions = pickle.load(file)